# Cats and dogs clasificator

Details

* Binary Classification problem
* Colored Images
* No Standard Dimension
* Kaggle Dataset > https://drive.google.com/open?id=16AlwTDOeyFaiP3RPxKOk5s80IycK80X4
* We will use subset of this data 



Data

Train (2000 images)
*   class_a (1000 images)
*   class_b (1000 images)
---
Valid
*   class_a (500 images)
*   class_b (500 images)
---
Test
*   Test_folder (1000 images)


In [12]:
# libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [13]:
# training directories
train_dir = "/content/drive/MyDrive/Colab Notebooks/cats_and_dogs_small/train"
validation_dir = "/content/drive/MyDrive/Colab Notebooks/cats_and_dogs_small/validation"
test_dir = "/content/drive/MyDrive/Colab Notebooks/cats_and_dogs_small/test"

** Data preprocessing **
* Read the picture files.
* Decode the JPEG content to RGB grids of pixels.
* Convert these into floating point tensors.
* Rescale the pixel vales (between 0 and 255) to the [0, 1] interval.

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
#Link : https://keras.io/preprocessing/image/

In [16]:
# Generating batches of tensor image data
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')


Found 2002 images belonging to 2 classes.
Found 2002 images belonging to 2 classes.


In [17]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [18]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))

model.add(layers.MaxPooling2D((2 ,2))) # After this step we will have images of 75 x 75

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 17, 17, 128)     

In [20]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
100/100 [==============================] - 1111s 11s/step - loss: 0.6915 - acc: 0.5288 - val_loss: 0.6771 - val_acc: 0.5640
Epoch 2/20
100/100 [==============================] - 111s 1s/step - loss: 0.6598 - acc: 0.6044 - val_loss: 0.6934 - val_acc: 0.5420
Epoch 3/20
100/100 [==============================] - 111s 1s/step - loss: 0.6135 - acc: 0.6569 - val_loss: 0.5781 - val_acc: 0.6810
Epoch 4/20
100/100 [==============================] - 112s 1s/step - loss: 0.5662 - acc: 0.7069 - val_loss: 0.5371 - val_acc: 0.7300
Epoch 5/20
100/100 [==============================] - 111s 1s/step - loss: 0.5433 - acc: 0.7230 - val_loss: 0.5592 - val_acc: 0.6920
Epoch 6/20
100/100 [==============================] - 111s 1s/step - loss: 0.5163 - acc: 0.7316 - val_loss: 0.5045 - val_acc: 0.7400
Epoch 7/20
100/100 [==============================] - 111s 1s/step - loss: 0.4992 - acc: 0.7538 - val_loss: 0.4976 - val_acc: 0.7450
Epoch 8/20
100/100 [==============================] - 111s 1s/step 

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
model.save("model.h5")

In [ ]:
from tensorflow.keras import backend as k

k.clear_session()
del model